In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import re
import pickle
from selenium.webdriver.common.keys import Keys

In [2]:
file_path = 'C:\\Users\\qkfka\\Desktop\\dd\\cafe\\' # 파일 경로 바꿀것
path = file_path + 'chromedriver.exe' #크롬드라이버
driver = webdriver.Chrome(path)

In [3]:
def gett(www):
    driver.get('{}'.format(www))

In [4]:
title_list = []
url_list = []

def urll():
    url_list.clear()
    title_list.clear()
    try :
        for j in range(0,100):
            for i in range(0,10):
                try : 
                    title = driver.find_element_by_xpath('//*[@id="cafeResultUL"]/li[{}]/div[2]/div/div[1]/a'.format(i+1)).text
                except : 
                    title = driver.find_element_by_xpath('//*[@id="cafeResultUL"]/li[{}]/div/div/div[1]/a'.format(i+1)).text

                try :
                    for_url = driver.find_element_by_xpath('//*[@id="cafeResultUL"]/li[{}]/div[2]/div/div[1]/a'.format(i+1))
                    url = for_url.get_attribute('href')
                except:
                    for_url = driver.find_element_by_xpath('//*[@id="cafeResultUL"]/li[{}]/div/div/div[1]/a'.format(i+1))
                    url = for_url.get_attribute('href')

                title_list.append(title)
                url_list.append(url)
                time.sleep(0.5)
            if j%10 ==0 :
                print(j)
            try:
                driver.find_element_by_xpath('//*[@id="pagingArea"]/span/span[3]/a').click()
            except:
                pass
    except : 
        print("완료 ")
        print(j*10 + i)

In [5]:
doc_list = []

def tab(url_list):
    doc_list.clear()
    for i in range(len(url_list)):
        url_path = url_list[i]
        driver.switch_to_window(driver.window_handles[0]) # 탭 0번 전환
        driver.execute_script("window.open('{}')".format(url_path))
        driver.switch_to_window(driver.window_handles[1]) # 탭 1번 전환 
        time.sleep(5)

        driver.switch_to_frame(0) # 첫번째 iframe
        html = driver.page_source
        soup = BS(html, 'html.parser')
        a = soup.find('div', class_='tx-content-container').get_text() 
        a = a.replace("\t", "")
        doc_list.append(a.replace("\n", ""))
        driver.switch_to_default_content()
        driver.close()
        time.sleep(3)
    return doc_list

In [ ]:
title_list = []
url_list = []

for j in range(0,8):
    for i in range(1,16):
        title = driver.find_element(By.XPATH,'//*[@id="twcColl"]/div[2]/c-container/c-card[{}]/div/c-doc-web/div/div[1]'.format(i)).text
        title2 = driver.find_element(By.XPATH, '//*[@id="twcColl"]/div[2]/c-container/c-card[{}]/div/c-doc-web/div/div[1]/c-title'.format(i))
        url = title2.get_attribute('data-href')
    
        title_list.append(title)
        url_list.append(url)
    driver.find_element(By.CLASS_NAME, 'btn_next')

In [7]:
def save_pkl(query_txt): #파일 저장
    raw_data = pd.DataFrame()
    raw_data['title'] = title_list
    raw_data['doc'] = doc_list
    raw_data['url'] = url_list
    raw_data['ch'] = 'daum'
    raw_data['ch2'] = 'cafe'
    
    
    f = open(file_path + "{}.pkl".format(query_txt), "wb")
    pickle.dump(raw_data, f)
    f.close()

In [13]:
gett('https://search.daum.net/search?w=cafe&nil_search=btn&DA=STC&enc=utf8&ASearchType=1&lpp=10&rlang=0&q=대전 지역경제&sd=20180101000000&ed=20201231235959&period=u')
urll()
len(url_list)
tab(url_list)
save_pkl('daum_cafe_1')
driver.quit()